# Bronze Layer: Initial Load & Sanity Check
**Purpose:** Confirm that all raw CSVs were ingested into Bronze Delta tables and are queryable.  
**Inputs:** Imported CSV files → saved as `bronze_customers`, `bronze_orders`, `bronze_order_items`, `bronze_products`.  
**Outputs:** Row counts + first 5 rows preview for each Bronze table.  

**Key steps:**
- Set active catalog and schema (`workspace.default`).
- Loop through each Bronze table name.
- Print row count to verify successful load.
- Display a sample of 5 rows for a quick structure check.  

**Why it matters:** Ensures the Bronze layer is ready before applying any transformations in Silver. It’s the foundation of the Medallion pipeline.

In [0]:
# Switch to catalog/schema
spark.sql("USE CATALOG workspace")
spark.sql("USE SCHEMA default")

tables = ["bronze_customers", "bronze_orders", "bronze_order_items", "bronze_products"]

for t in tables:
    print(f"Table: {t}")
    spark.sql(f"SELECT COUNT(*) as rows FROM {t}").show()
    spark.sql(f"SELECT * FROM {t} LIMIT 5").show()

Table: bronze_customers
+----+
|rows|
+----+
|  41|
+----+

+-----------+-------------+------+-----------+
|customer_id|customer_name|region|signup_date|
+-----------+-------------+------+-----------+
|          1| Lucas Garcia|    WA| 2022-05-23|
|          2|Oliver Martin|    SA| 2022-06-13|
|          3|    Leo Brown|   ACT| 2023-07-24|
|          4|    Zoe Kumar|   QLD| 2024-01-29|
|          5|  Lucas Jones|    SA| 2022-05-27|
+-----------+-------------+------+-----------+

Table: bronze_orders
+----+
|rows|
+----+
| 121|
+----+

+--------+-----------+----------+---------+
|order_id|customer_id|order_date|   status|
+--------+-----------+----------+---------+
|       1|          7|2024-03-03|CANCELLED|
|       2|         22|2024-04-21|  Shipped|
|       3|         13|2024-07-13| Returned|
|       4|         29|2024-05-23|CANCELLED|
|       5|         12|2024-10-12|CANCELLED|
+--------+-----------+----------+---------+

Table: bronze_order_items
+----+
|rows|
+----+
| 312|
+----+



# 🥈 Silver Layer: Cleaned & Standardized Data
The Silver layer refines raw Bronze data into **typed, de-duplicated, and normalized tables**.  
This ensures consistent joins and analytics downstream.  
Key activities in Silver:  
- Casting IDs to correct types (INT, DOUBLE).  
- Trimming text fields and normalizing case.  
- Replacing blanks with NULLs where appropriate.  
- Removing duplicate rows on natural keys.  

Silver tables serve as the **single source of truth** for building the Gold layer.

## Silver: Customers
**Purpose:** Standardize and clean customer attributes for reliable analytics.  
**Inputs:** `bronze_customers`  
**Output:** `silver_customers` (Delta table)

**Key steps:**
- Cast `customer_id` → INT  
- Trim whitespace in `customer_name`  
- Normalize `region` to uppercase for consistency  
- Keep `signup_date` as STRING for now (parse later if needed)  
- Drop duplicates on `customer_id`  

**Why it matters:** Provides a clean customer dimension table that can be safely joined with orders and order_items in the Gold layer.

In [0]:
from pyspark.sql import functions as F

spark.sql("USE CATALOG workspace")
spark.sql("USE SCHEMA default")

bronze = spark.table("bronze_customers")

silver_customers = (
    bronze
    .withColumn("customer_id", F.col("customer_id").cast("int"))
    .withColumn("customer_name", F.trim(F.col("customer_name")))
    .withColumn("region", F.upper(F.trim(F.col("region"))))
    # keeping signup_date as string for now 
    .dropDuplicates(["customer_id"])
)

(
    silver_customers.write
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .format("delta")
    .saveAsTable("silver_customers")
)

display(spark.table("silver_customers").limit(10))

customer_id,customer_name,region,signup_date
29,Ava Davis,ACT,2023-03-10
24,Ethan Nguyen,VIC,14/01/2024
39,Noah Brown,ACT,2023-12-27
38,Leo Kim,ACT,2023-01-25
15,Oliver Brown,QLD,2023/13/40
20,Henry Chen,QLD,2024-07-18
21,aarav thompson,QLD,2024-11-05
22,Noah White,WA,2022-11-10
26,Emily Williams,VIC,2024-09-22
17,Grace Davis,NT,2024-12-19


## Silver: Orders
**Purpose:** Standardize and clean order-level data for consistent downstream joins.  
**Inputs:** `bronze_orders`  
**Output:** `silver_orders` (Delta table)  

**Key steps:**
- Cast `customer_id` and `order_id` → INT  
- Normalize `status` → trimmed + uppercase, blanks replaced with NULL  
- Trim `order_date` and keep as STRING (date parsing handled later in Gold)  
- Drop duplicates on `order_id`  

**Why it matters:** Produces a consistent orders table that links customers to their transactions.  
It also preserves order_date for parsing while ensuring status values are clean for analysis.

In [0]:
from pyspark.sql import functions as F

spark.sql("USE CATALOG workspace")
spark.sql("USE SCHEMA default")

bronze = spark.table("bronze_orders")

silver_orders = (
    bronze
    .withColumn("customer_id", F.col("customer_id").cast("int"))
    .withColumn("order_id", F.col("order_id").cast("int"))
    .withColumn("status", F.upper(F.trim(F.col("status"))))
    .withColumn("status", F.when(F.col("status") == "", F.lit(None)).otherwise(F.col("status")))
    .withColumn("order_date",  F.trim(F.col("order_date")))   # keep as string for now
    .dropDuplicates(["order_id"])
)

(
    silver_orders.write
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .format("delta")
    .saveAsTable("silver_orders")
)

display(spark.table("silver_orders").limit(10))

order_id,customer_id,order_date,status
94,27,2024-07-18,CANCELLED
29,41,2024-11-03,CANCELLED
88,6,2024-10-17,RETURNED
108,5,2024-10-19,SHIPPED
56,27,2024-06-07,SHIPPED
24,16,2024-02-10,RETURNED
99,36,2024-09-12,PLACED
39,9,2024-09-27,PLACED
102,21,03/29/2025,RETURNED
38,36,2024-11-02,PLACED


## Silver: Order Items
**Purpose:** Clean and standardize order line-item data for accurate revenue calculations.  
**Inputs:** `bronze_order_items`  
**Output:** `silver_order_items` (Delta table)  

**Key steps:**
- Cast IDs (`order_item_id`, `product_id`, `order_id`) → INT  
- Cast `quantity` → INT, and filter out zero or negative quantities  
- Cast `unit_price` → DOUBLE  
- Drop duplicates on `order_item_id`  

**Why it matters:** Creates a reliable order-items table that can be joined with products and orders in Gold.  
Filtering invalid quantities ensures downstream KPIs like revenue are not distorted.

In [0]:
from pyspark.sql import functions as F

spark.sql("USE CATALOG workspace")
spark.sql("USE SCHEMA default")

bronze = spark.table("bronze_order_items")

silver_order_items = (
    bronze
    .withColumn("order_item_id", F.col("order_item_id").cast("int"))
    .withColumn("product_id", F.col("product_id").cast("int"))
    .withColumn("order_id", F.col("order_id").cast("int"))
    .withColumn("quantity", F.col("quantity").cast("int"))
    .filter(F.col("quantity") > 0)  # drop zero/negative
    .withColumn("unit_price", F.col("unit_price").cast("double"))
    .dropDuplicates(["order_item_id"])
)

(
    silver_order_items.write
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .format("delta")
    .saveAsTable("silver_order_items")
)

display(spark.table("silver_order_items").limit(10))

order_item_id,order_id,product_id,quantity,unit_price
94,36,6,1,20.73
29,14,19,1,424.51
284,107,2,4,141.14
88,34,2,5,141.14
131,51,2,4,141.14
148,58,2,4,141.14
279,104,14,3,84.03
190,73,16,1,111.58
195,75,14,2,84.03
56,23,7,3,113.23


## Silver: Products
**Purpose:** Standardize and clean product catalog data for consistent use in analytics.  
**Inputs:** `bronze_products`  
**Output:** `silver_products` (Delta table)  

**Key steps:**
- Cast `product_id` → INT  
- Trim `product_name`, replace blanks with NULL  
- Normalize `category` → uppercase, replace blanks with NULL  
- Clean `list_price` by removing `$` and commas, then cast to DOUBLE  
- Drop duplicates on `product_id`  

**Why it matters:** Produces a clean, standardized product dimension that links order items to categories and list prices, enabling reliable revenue breakdowns by product and category.

In [0]:
from pyspark.sql import functions as F

spark.sql("USE CATALOG workspace")
spark.sql("USE SCHEMA default")

bronze = spark.table("bronze_products")

silver_products = (
    bronze
    .withColumn("product_id", F.col("product_id").cast("int"))

    .withColumn("product_name", F.trim(F.col("product_name")))
    .withColumn("product_name",F.when(F.col("product_name") == "", F.lit(None)).otherwise(F.col("product_name")))

    .withColumn("category", F.upper(F.trim(F.col("category"))))
    .withColumn("category", F.when(F.col("category") == "", F.lit(None)).otherwise(F.col("category")))

    .withColumn("list_price", F.regexp_replace(F.col("list_price").cast("string"), r"[$,]", ""))
    .withColumn("list_price", F.col("list_price").cast("double"))
    .dropDuplicates(["product_id"])
)

(
    silver_products.write
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .format("delta")
    .saveAsTable("silver_products")
)

display(spark.table("silver_products").limit(10))

product_id,category,product_name,list_price
24,BOOKS,Books Item 24,415.56
15,TOYS,Toys Item 15,173.42
20,CLOTHING,Clothing Item 20,135.94
21,ELECTRONICS,Electronics Item 21,366.22
22,CLOTHING,Clothing Item 22,66.79
17,BEAUTY,Beauty Item 17,171.61
1,SPORTS,Sports Item 1,60.11
23,TOYS,Toys Item 23,44.01
4,ELECTRONICS,Electronics Item 4,339.97
13,BEAUTY,Beauty Item 13,8.22


# ✨ Gold Layer: Business-Ready Data
The Gold layer transforms cleaned Silver data into **fact and dimension tables** that follow a star-schema style model.  
This layer is optimized for analytics and reporting.  

Key activities in Gold:  
- Building **dimension tables** (customers, products) that hold descriptive attributes.  
- Building **fact tables** (order lines, daily sales) that store measurable events and metrics.  
- Parsing and standardizing dates for time-based analysis.  
- Adding calculated business metrics like revenue and average order value (AOV).  

Gold tables are what business intelligence tools (Power BI, Looker Studio, etc.) connect to directly.

In [0]:
spark.sql("USE CATALOG workspace")
spark.sql("USE SCHEMA default")
spark.sql("CREATE SCHEMA IF NOT EXISTS gold")

DataFrame[]

## Gold: dim_customer
**Purpose:** Create a curated customer dimension for joining with facts.  
**Inputs:** `silver_customers`  
**Output:** `gold.dim_customer`  

**Key steps:**
- Select only required attributes: `customer_id`, `customer_name`, `region`, `signup_date`  
- Exclude rows with NULL `customer_id`  
- Keep `signup_date` as STRING (no parsing applied yet)  

**Why it matters:** Provides a consistent customer lookup table to enrich fact tables with descriptive attributes like name and region.

In [0]:
spark.sql("""
CREATE OR REPLACE TABLE gold.dim_customer AS
SELECT
  customer_id,
  customer_name,
  region,
  signup_date   -- kept as string per Silver
FROM silver_customers
WHERE customer_id IS NOT NULL
""")

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

## Gold: dim_product
**Purpose:** Build the product dimension for enriching sales facts with descriptive attributes.  
**Inputs:** `silver_products`  
**Output:** `gold.dim_product`  

**Key steps:**
- Select `product_id`, `product_name`, `category`, `list_price`  
- Exclude rows with NULL `product_id`  

**Why it matters:** Serves as a standardized product lookup table, enabling reporting by product name or category and supporting revenue analysis by product hierarchy.

In [0]:
spark.sql("""
CREATE OR REPLACE TABLE gold.dim_product AS
SELECT
  product_id,
  product_name,
  category,
  list_price
FROM silver_products
WHERE product_id IS NOT NULL
""")

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

## Gold: fact_order_item & fact_daily_sales
**Purpose:** Create the central fact tables that capture sales at the line-item level and the daily aggregate level.  
**Inputs:** `silver_order_items`, `silver_orders`, `silver_products`, `silver_customers`  
**Outputs:** 
- `gold.fact_order_item` (order-line grain fact)  
- `gold.fact_daily_sales` (daily-grain aggregate fact)  

**Key steps:**
1. Force `order_date` to STRING, then safely parse into `order_date_parsed` by handling multiple formats (ISO, US, EU).  
2. Join orders, order_items, products, and customers into a single fact table.  
3. Select key fields: IDs, raw & parsed dates, quantity, unit_price, and derived `line_amount`.  
4. Write `fact_order_item` to Gold as a Delta table.  
5. Build `fact_daily_sales` by grouping on `order_date_parsed` and calculating:  
   - Orders count (`orders_cnt`)  
   - Items count (`items_cnt`)  
   - Total revenue (`revenue`)  
   - Distinct customers (`distinct_customers`)  
   - Average order value (`aov = revenue / orders_cnt`)  

**Why it matters:**  
This fact structure enables answering key business questions like:  
- *What is our daily revenue trend?*  
- *Who are the top customers by spend?*  
- *Which product categories generate the most sales?*  

In [0]:
from pyspark.sql import functions as F

spark.sql("USE CATALOG workspace")
spark.sql("USE SCHEMA default")
spark.sql("DROP TABLE IF EXISTS gold.fact_daily_sales")

# 1) Load Silver tables
i = spark.table("silver_order_items")
o = spark.table("silver_orders")
p = spark.table("silver_products")
c = spark.table("silver_customers")

# 2) Force order_date to STRING (avoids hidden types) and trim
o = o.withColumn("order_date", F.trim(F.col("order_date").cast("string")))

# 3) Safe parse: only call to_date when the string matches a known pattern
#    - ISO: YYYY-MM-DD
#    - US:   MM/DD/YYYY
#    - EU:   DD/MM/YYYY
o2 = (
    o.withColumn(
        "order_date_parsed",
        F.when(F.col("order_date").rlike(r'^\d{4}-\d{2}-\d{2}$'),
               F.to_date(F.col("order_date"), "yyyy-MM-dd"))
         .when(F.col("order_date").rlike(r'^\d{2}/\d{2}/\d{4}$') & F.col("order_date").substr(1,2).rlike(r'^(0[1-9]|1[0-2])$'),
               # If month-first (MM/DD/YYYY): parse as US
               F.to_date(F.col("order_date"), "MM/dd/yyyy"))
         .when(F.col("order_date").rlike(r'^\d{2}/\d{2}/\d{4}$'),
               # Otherwise treat as DD/MM/YYYY
               F.to_date(F.col("order_date"), "dd/MM/yyyy"))
         .otherwise(F.lit(None).cast("date"))
    )
)

# 4) Build fact_order_item from PySpark 
fact = (
    i.join(o2, "order_id", "inner")
     .join(p,  "product_id", "inner")
     .join(c,  "customer_id", "inner")
     .select(
         i.order_item_id,
         i.order_id,
         o2.customer_id,
         i.product_id,
         o2.order_date.alias("order_date_raw"),
         o2.order_date_parsed,
         i.quantity,
         i.unit_price,
         (i.quantity * i.unit_price).alias("line_amount")
     )
)

# 5) Write to a new gold table 
(fact.write
     .mode("overwrite")
     .option("overwriteSchema","true")
     .format("delta")
     .saveAsTable("gold.fact_order_item"))

# 6) Build daily aggregate 
f = spark.table("gold.fact_order_item")
daily = (
    f.where(F.col("order_date_parsed").isNotNull())
     .groupBy("order_date_parsed")
     .agg(
         F.countDistinct("order_id").alias("orders_cnt"),
         F.count("*").alias("items_cnt"),
         F.sum("line_amount").alias("revenue"),
         F.countDistinct("customer_id").alias("distinct_customers")
     )
     .withColumn("aov", F.round(F.col("revenue") / F.when(F.col("orders_cnt") != 0, F.col("orders_cnt")), 2))
)

(daily.write
     .mode("overwrite")
     .option("overwriteSchema","true")
     .format("delta")
     .saveAsTable("gold.fact_daily_sales"))

# 7) Inspect results
display(spark.table("gold.fact_order_item").limit(10))
display(spark.table("gold.fact_daily_sales").orderBy("order_date_parsed").limit(20))


order_item_id,order_id,customer_id,product_id,order_date_raw,order_date_parsed,quantity,unit_price,line_amount
94,36,29,6,2025-07-10,2025-07-10,1,20.73,20.73
29,14,28,19,2025-07-23,2025-07-23,1,424.51,424.51
284,107,16,2,2024-10-08,2024-10-08,4,141.14,564.56
88,34,23,2,03/11/2024,2024-03-11,5,141.14,705.6999999999999
131,51,24,2,07/19/2025,2025-07-19,4,141.14,564.56
279,104,29,14,2024-06-11,2024-06-11,3,84.03,252.09
190,73,37,16,2024-07-13,2024-07-13,1,111.58,111.58
195,75,34,14,2025-07-05,2025-07-05,2,84.03,168.06
56,23,26,7,2024/05/02,null,3,113.23,339.69
172,67,26,18,2025-07-03,2025-07-03,2,193.06,386.12


order_date_parsed,orders_cnt,items_cnt,revenue,distinct_customers,aov
2024-01-01,1,1,96.06,1,96.06
2024-01-03,1,2,1850.16,1,1850.16
2024-01-06,1,1,173.42,1,173.42
2024-01-16,1,2,809.52,1,809.52
2024-02-10,1,3,943.24,1,943.24
2024-02-21,1,3,1342.96,1,1342.96
2024-02-22,1,2,213.79,1,213.79
2024-02-28,1,4,3432.5400000000004,1,3432.54
2024-02-29,1,2,1353.8600000000001,1,1353.86
2024-03-01,1,2,258.91,1,258.91


# 📊 KPIs & Quick Dashboard
With Gold fact and dimension tables in place, we can now run business-facing queries.  
These KPIs demonstrate the value of the Medallion pipeline: turning raw CSVs into actionable insights.  
Results from these queries can also be exported into BI tools (Power BI, Looker Studio) for visualization dashboards.

## KPI: Top 10 Customers by Revenue
**Purpose:** Identify which customers contribute the most to total sales.  
**Inputs:** `gold.fact_order_item`, `gold.dim_customer`  
**Output:** Ranked list of top 10 customers by total revenue.  

**Why it matters:** Provides visibility into high-value customers and supports customer segmentation or loyalty strategies.

In [0]:
%sql
-- Top 10 customers by revenue
SELECT c.customer_id, c.customer_name, ROUND(SUM(f.line_amount),2) AS revenue
FROM gold.fact_order_item f
JOIN gold.dim_customer c USING (customer_id)
GROUP BY c.customer_id, c.customer_name
ORDER BY revenue DESC
LIMIT 10;

customer_id,customer_name,revenue
16,Liam Davis,10584.52
7,zoe singh,9188.6
6,Aria Wilson,8114.65
18,Mia Hernandez,7330.56
21,aarav thompson,6913.42
24,Ethan Nguyen,6908.64
29,Ava Davis,6603.07
11,Ethan Singh,6105.44
27,Charlotte Hernandez,5839.05
10,Amelia Garcia,5824.43


## KPI: Revenue by Product Category
**Purpose:** Show how sales are distributed across product categories.  
**Inputs:** `gold.fact_order_item`, `gold.dim_product`  
**Output:** Aggregated revenue per product category, sorted highest to lowest.  

**Why it matters:** Highlights which categories drive the most revenue, supporting decisions on inventory, promotions, and strategic focus.

In [0]:
%sql
-- Revenue by product category
SELECT p.category, ROUND(SUM(f.line_amount),2) AS revenue
FROM gold.fact_order_item f
JOIN gold.dim_product p USING (product_id)
GROUP BY p.category
ORDER BY revenue DESC;

category,revenue
CLOTHING,43233.93
ELECTRONICS,40590.67
BOOKS,31533.95
SPORTS,22044.79
TOYS,14204.81
BEAUTY,7038.58
HOME & KITCHEN,6269.84


Databricks visualization. Run in Databricks to view.

## Export Gold Data for Dashboarding
**Purpose:** Make the curated Gold tables available outside Databricks for visualization in BI tools (e.g., Power BI, Looker Studio).  
**Steps:**  
- Select the target Gold table (`gold.fact_daily_sales` in this case).  
- Use `display(df)` to preview results.  
- Download the data as CSV directly from the notebook UI.  

**Why it matters:** Provides a portable dataset for building quick dashboards and sharing business insights beyond Databricks.

In [0]:
# Pick your gold table, gold.fact_daily_sales in this case, and download as CSV to import in the dashboard.
df = spark.table("gold.fact_daily_sales")
display(df)

order_date_parsed,orders_cnt,items_cnt,revenue,distinct_customers,aov
2024-11-04,1,2,564.49,1,564.49
2025-05-03,1,1,415.56,1,415.56
2024-03-10,1,3,4242.59,1,4242.59
2024-10-10,1,4,2540.6,1,2540.6
2024-03-03,2,3,949.7700000000001,2,474.89
2024-10-12,2,3,1073.26,2,536.63
2024-09-12,1,4,3904.7400000000002,1,3904.74
2024-07-16,1,1,772.24,1,772.24
2025-07-20,1,2,2249.38,1,2249.38
2024-04-21,1,4,2314.1400000000003,1,2314.14


# 📂 Portfolio Summary

This project showcases a complete **end-to-end data engineering pipeline** built in Databricks using the **Medallion architecture**.  
Starting with raw e-commerce CSV files (Bronze), I cleaned and standardized them into curated Delta tables (Silver), and finally modeled them into fact and dimension tables (Gold).  

With this foundation, I produced key KPIs such as **Top 10 Customers by Revenue**, **Revenue by Product Category**, and a **Daily Sales Trend**, and exported the results for BI dashboarding.  

This work highlights my ability to:  
- Ingest and transform raw data into structured layers.  
- Apply **PySpark transformations** and safe parsing techniques.  
- Design **fact/dimension tables** for analytics.  
- Deliver business-ready insights that can be consumed directly in dashboards.  

✅ Demonstrates both **technical skills (PySpark, Delta, Databricks)** and the ability to connect data pipelines to **real business value**.